I loved simple bank. It was the easies banking interface ever. And it geared to help me save money not spend it, lol.

Sadly, they're gone. So, here's my initial crack at making one.

In [21]:
import pandas as pd
import numpy as np
#import datetime as dt     
today = pd.to_datetime('today').normalize()

In [22]:
# working functions here

def create_date_table(start='2000-01-01', end='2050-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Day_Name"] = df.Date.dt.day_name()
    # new column
    df["Day"] = df.Date.dt.day
    df["Week"] = df.Date.dt.weekofyear
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    return df

def calculate_paydays(df_cal, payday = '2021-08-25', frequency = 2, income = 1200):
    #payday = input("When's your next payday? ")
    #frequency = input("How often do you get paid? (enter 1 for weekly, 2 for bi-weekly, 3 for monthy, 4 for twice monthly) ")
    #df_cal['payday'] = np.where(df_cal['Date'] == payday, 'payday', '')
    #income = input("How much money do you get each payday? ")
    start = df_cal.index[df_cal['Date'] == payday].tolist()
    paydays = df_cal[df_cal.index % 14 == start[0]].index
    df_cal['payday'] = np.where(df_cal.index.isin(paydays), 'payday', np.NaN)
    df_cal['income'] = np.where(df_cal.index.isin(paydays), income, np.NaN)
    df_cal['set_aside'] = np.NaN
    
# what on earth is this function doing....
def set_aside_all(goals, bills, df_cal):
    goals['save_per_day'] = np.NaN
    for index, row in goals.iterrows():
        due_date = row['due_date']
        paydays_between = df_cal[(df_cal.payday == 'payday')
                                 & (df_cal.Date > today)
                                 & (df_cal.Date < due_date)
                                ].payday.count()
        savings_per_payday = row['amount'] / paydays_between
        #print(savings_per_payday)
        #print(index)
        goals.loc[index,'save_per_day'] = savings_per_payday

    bills['save_per_day'] = np.NaN
    for index, row in bills.iterrows():
        due_date = row['due_date']
        paydays_between = df_cal[(df_cal.payday == 'payday')
                                 & (df_cal.Date > today)
                                 & (df_cal.Date < due_date)
                                ].payday.count()
        savings_per_payday = row['amount'] / paydays_between
        #print(savings_per_payday)
        #print(index)
        bills.loc[index,'save_per_day'] = savings_per_payday

    # now take the goals data-frame complete with 
    for index, row in df_cal.iterrows():
        if row['payday'] == 'payday':
            date = row['Date']
            set_aside = goals[['due_date'
                               , 'save_per_day']].sum(axis=1).where(goals['due_date'] > date, 0).sum() + bills[['due_date'
                       , 'save_per_day'
                      ]].sum(axis=1).where(bills['due_date'].dt.day > row['Day'], 0).sum()
            df_cal.loc[index, 'set_aside'] = set_aside
        else: pass

In [23]:
df_cal = create_date_table('2021-08-01', '2021-12-31')

<ipython-input-22-06a5a27b3736>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["Week"] = df.Date.dt.weekofyear


In [ ]:
calculate_paydays(df_cal)

In [ ]:
goals = pd.read_excel('input_list.xlsx', sheet_name = 'goals')
bills = pd.read_excel('input_list.xlsx', sheet_name = 'bills')

In [ ]:
set_aside_all(goals, bills, df_cal)

In [ ]:
df_cal[df_cal['payday']!='nan']